# CONSTITUTION OF KENYA RAG

In [1]:
%pip install langchain-groq langchain sentence-transformers langchain-community  pypdf chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.9/131.9 kB 9.7 MB/s eta 0

In [2]:
#importing libraries
from sentence_transformers import SentenceTransformer
from langchain.retrievers import ParentDocumentRetriever
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.storage import InMemoryStore
from langchain_groq import ChatGroq




In [3]:
doc_path="/content/The_Constitution_of_Kenya_2010.pdf"
loader= PyPDFLoader(doc_path)
docs=loader.load()
print(len(docs))

193


In [4]:
#initializing parent and text splitters
parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=100,
)

child_splitter = RecursiveCharacterTextSplitter(
    chunk_size=700,
    chunk_overlap=50,
)


In [5]:
#initializing the embedding model
bge_model=SentenceTransformer("BAAI/bge-base-en-v1.5")
class BGEEmbeddings:
  def embed_documents(self, text):
    return bge_model.encode(text,batch_size=8,normalize_embeddings=True).tolist()
  def embed_query(self, query):
    return bge_model.encode([query],normalize_embeddings=True).tolist()[0]



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
#creating the vector store
constitution_vector_store=Chroma(
    collection_name="constitution",
    embedding_function=BGEEmbeddings(),
    persist_directory="./chroma",
)



In [8]:
#creating in-memory store
store=InMemoryStore()

In [9]:
#creating retrievers
retriever=ParentDocumentRetriever(
    vectorstore=constitution_vector_store,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [10]:
#adding documents into the vector store
retriever.add_documents(docs)

In [12]:
#creating a child retriever
child_retriever=constitution_vector_store.as_retriever()

In [13]:
#initializing the LLM
llm=ChatGroq(
   groq_api_key=userdata.get('constitution_of_kenya'),
   model_name="llama-3.3-70b-versatile",
   temperature=0.7

)

In [14]:
#creating the template
template="""
 Answer the following questions according to the context given.
 If the question is out of context,just say the question is out of context Kindly give me another question.
 Do not try to answer a question out of context.
 {context}
 Question: {question}
"""


In [15]:
#creating the chain
qa_chain=PromptTemplate(template=template,input_variables=["context","question"])

In [16]:
chain=RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt":qa_chain},
    return_source_documents=True
)

In [18]:
question="How many levels of government are established under the Constitution of Kenya 2010?"
answer=chain({"query":question})
print(answer["result"])

There are 2 levels of government established under the Constitution of Kenya 2010: 

1. National government 
2. County government


In [19]:
question="What does Article 27 say about equality and freedom from discrimination?"
answer=chain({"query":question})
print(answer["result"])

Article 27 of the Constitution of Kenya, 2010, states that every person is equal before the law and has the right to equal protection and equal benefit of the law. It emphasizes that equality includes the full and equal enjoyment of all rights and fundamental freedoms. 

The article also highlights that women and men have the right to equal treatment, including the right to equal opportunities in political, economic, cultural, and social spheres. 

Additionally, it prohibits the State from discriminating directly or indirectly against any person on various grounds, including race, sex, pregnancy, marital status, health status, ethnic or social origin, color, age, disability, religion, conscience, belief, culture, dress, language, or birth. 

It also states that a person shall not discriminate directly or indirectly against another person on any of the grounds specified or contemplated in the article.
